# Geração de bases para modelo para análise exploratória

Basicamente as premissas aplicadas para limpeza dos dados foram:
- Retirada linhas que possuam valores nulos em quaisquer dos campos a seguir: `GESTACAO`, `GRAVIDEZ`, `PARTO`, `CONSULTAS`, `ESCMAE`,`QTDFILVIVO`, `QTDFILMORT`.
- Campo `IDADEMAE` abaixo de 70 anos (revisar).
- Campos `GESTACAO`, `GRAVIDEZ`, `PARTO`, `CONSULTAS`, `ESCMAE` e `SEXO` contendo dados diferentes de valores ignorados ou zero.
- Campos `QTDFILVIVO` e `QTDFILMORT` abaixo de 99.

O dicionário de campos da base de dados do SINASC se encontra nesta [planilha](https://docs.google.com/spreadsheets/d/1TwysowkLCLIGGLplKkKibahrsG52NqVKOZuTjWyrZ-8/edit#gid=1412172863).

# 📚 Libs

In [1]:
import pandas as pd
import numpy as np
import re #para expressões regulares
from datetime import datetime

# 🎲 FATO SINASC

## Importação

In [2]:
dados = pd.read_csv("DNOPEN22.csv", sep=";")

C:\Users\carol\AppData\Local\Temp\ipykernel_29228\2318329602.py:1: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv("DNOPEN22.csv", sep=";")


## Limpeza dos dados

In [3]:
#mantendo colunas que vamos usar
colunas_selecionadas = ['DTNASC','LOCNASC','CODMUNNASC','CODESTAB','SEXO','RACACOR','PESO','CODANOMAL','IDANOMAL','GESTACAO','SEMAGESTAC','GRAVIDEZ','TPAPRESENT','STTRABPART','STCESPARTO','PARTO','TPNASCASSI','IDADEMAE','ESTCIVMAE','RACACORMAE','ESCMAE','ESCMAE2010','ESCMAEAGR1','CODOCUPMAE','PARIDADE','QTDGESTANT','QTDFILVIVO','QTDFILMORT','QTDPARTNOR','QTDPARTCES','MESPRENAT','CONSULTAS','CONSPRENAT','KOTELCHUCK','TPROBSON','IDADEPAI']
dados = dados[colunas_selecionadas]

In [4]:
#retirando linhas mediante condições
dados = dados[(dados.IDADEMAE <= 70.0)]
dados = dados[(dados.GESTACAO < 9.0)]
dados = dados[(dados.GRAVIDEZ < 9.0)]
dados = dados[(dados.PARTO < 9.0)]
dados = dados[(dados.CONSULTAS < 9.0)]
dados = dados[(dados.ESCMAE < 9.0)]
dados = dados[(dados.QTDFILVIVO < 99.0)]
dados = dados[(dados.QTDFILMORT < 99.0)]
dados = dados[(dados.SEXO > 0)]

## Transformação de colunas

### DTNASC

In [5]:
dados['DTNASC'] = dados['DTNASC'].astype(str).str.pad(8, side = 'left', fillchar = '0')
dados['DTNASC'] = pd.to_datetime(dados['DTNASC'], format='%d%m%Y')

### CODANOMAL

In [6]:
#substitui a letra X por nada, pois não encontramos anomalia contendo a letra X no fim
dados['CODANOMAL'] = dados['CODANOMAL'].str.replace('X', '')
#substituindo o CID Q356 por Q359 pois foi substituído
#https://www.hidoctor.com.br/cid10/p/capitulo/17/grupo/Q35-Q37/categoria/Q35/subcategoria/Q359
dados['CODANOMAL'] = dados['CODANOMAL'].str.replace('Q356', 'Q359')
# conta quantas letras possui no campo CODANOMAL
dados['qt_anomal'] = dados['CODANOMAL'].str.count('[a-zA-Z]')
#substitui valores nulos e aplica formato de número inteiro
dados['qt_anomal'] = dados['qt_anomal'].fillna(0).astype(int)

### CODESTAB

In [7]:
dados['CODESTAB'] = dados['CODESTAB'].fillna(0).astype(int).astype(str)

## Criação de novas colunas

### indice

In [8]:
#coluna adicional de índice
dados['indice'] = range(1, len(dados) + 1)
dados['indice'] = dados['indice'].astype(str)

### ano_mes

In [9]:
dados['ano_mes'] = dados['DTNASC'].dt.strftime('%Y-%m')

### atualização colunas

In [10]:
#Renomeando todas colunas
novo_nome = {
    'DTNASC': 'dt_nasc',
    'LOCNASC': 'loc_nasc',
    'CODMUNNASC': 'cod_mun_nasc',
    'CODESTAB': 'cod_estab',
    'SEXO': 'sexo',
    'RACACOR': 'raca_cor',
    'PESO': 'peso',
    'CODANOMAL': 'cod_anomal',
    'IDANOMAL': 'id_anomal',
    'GESTACAO': 'gestacao',
    'SEMAGESTAC': 'sema_gestac',
    'GRAVIDEZ': 'gravidez',
    'TPAPRESENT': 'tpa_present',
    'STTRABPART': 'st_trab_parto',
    'STCESPARTO': 'st_ces_parto',
    'PARTO': 'parto',
    'TPNASCASSI': 'tp_nasc_assi',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'RACACORMAE': 'raca_cor_mae',
    'ESCMAE': 'esc_mae',
    'ESCMAE2010': 'esc_mae_2010',
    'ESCMAEAGR1': 'esc_mae_gr1',
    'CODOCUPMAE': 'cod_ocup_mae',
    'PARIDADE': 'paridade',
    'QTDGESTANT': 'qtd_gestant',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'QTDFILMORT': 'qtd_fil_mort',
    'QTDPARTNOR': 'qtd_part_nor',
    'QTDPARTCES': 'qtd_part_ces',
    'MESPRENAT': 'mes_pre_nat',
    'CONSULTAS': 'consultas',
    'CONSPRENAT': 'cons_pre_nat',
    'KOTELCHUCK': 'kotelchuck',
    'TPROBSON': 'tp_robson',
    'IDADEPAI': 'idade_pai'
}

dados.rename(columns=novo_nome, inplace=True)

In [11]:
#transformando colunas float que na verdade são string
colunas_float = ['raca_cor', 'cod_anomal', 'id_anomal', 'gestacao', 'gravidez', 'tpa_present', 'st_trab_parto', 'st_ces_parto', 'parto', 'tp_nasc_assi', 'est_civ_mae', 'raca_cor_mae', 'esc_mae', 'esc_mae_2010', 'cod_ocup_mae', 'qtd_gestant', 'qtd_fil_vivo', 'qtd_fil_mort', 'qtd_part_nor', 'qtd_part_ces', 'mes_pre_nat', 'consultas', 'cons_pre_nat', 'sema_gestac']

for coluna in colunas_float:
    dados [coluna] = dados[coluna].astype(str).str.replace('.0', '')

In [12]:
#transformando coluna que está como float mas é int
coluna_int = ['idade_mae', 'idade_pai']
for coluna in coluna_int:
    dados [coluna] = dados[coluna].fillna(0).astype(int)

In [13]:
#gerando planilha com colunas e formatos
dados_info = pd.DataFrame(dados.dtypes, columns=['Tipo'])
dados_info.index.name = 'Coluna'
dados_info = dados_info.reset_index()
dados_info.to_excel('formato_colunas_pos.xlsx', index=False)

In [14]:
dados.head()

Coluna,dt_nasc,loc_nasc,cod_mun_nasc,cod_estab,sexo,raca_cor,peso,cod_anomal,id_anomal,gestacao,...,qtd_part_ces,mes_pre_nat,consultas,cons_pre_nat,kotelchuck,tp_robson,idade_pai,qt_anomal,indice,ano_mes
0,2022-01-03,1,110001,2516500,2,1,3412.0,nan,2,5,...,0,2,4,7,5,7,20,0,1,2022-01
1,2022-01-07,1,110001,2516500,2,1,3594.0,nan,2,5,...,0,2,4,8,5,1,28,0,2,2022-01
2,2022-01-12,1,110001,2516500,2,nan,3470.0,nan,2,5,...,1,1,4,7,5,5,37,0,3,2022-01
3,2022-01-12,1,110001,2516500,2,4,3260.0,nan,2,5,...,1,1,4,31,5,5,40,0,4,2022-01
4,2022-01-17,1,110001,2516500,2,nan,3232.0,nan,2,5,...,0,2,4,8,5,2,37,0,5,2022-01


## csv

In [15]:
dados.to_csv('f_sinasc_tratado.csv', sep=';', index=False)

## Teste correspondência campo de estabelecimento

In [ ]:
#cod_estab_uni = pd.DataFrame({'cod_estab': dados['cod_estab'].unique()})

In [ ]:
#cod_estab_uni.shape

In [ ]:
#cod_estab_uni = pd.merge(cod_estab_uni, df_estab, left_on= 'cod_estab', right_on= 'co_cnes', how = 'left')

In [ ]:
#cod_estab_uni.shape

In [ ]:
#cod_estab_uni.info()
#3% sem info de latlong (falta checar se são válidas) e 1% de natureza não tem info

In [ ]:
#contagem = cod_estab_uni.groupby('classe_natureza').size().reset_index(name = 'qt_linhas')

In [ ]:
#contagem.sort_values(by='qt_linhas', ascending=False).head(10)

# 🎲 FATO ANOMALIAS

In [ ]:
df_cod_anomal_sep = pd.DataFrame({'cod_anomal': dados['cod_anomal'].unique()})
#quebra os códigos de anomalias
df_cod_anomal_sep['cod_anomal_sep'] =  df_cod_anomal_sep['cod_anomal'].str.findall(r'[a-zA-Z]+\d+')

In [ ]:
df_cod_anomal_sep = df_cod_anomal_sep.explode('cod_anomal_sep')

In [ ]:
df_cod_anomal_sep = df_cod_anomal_sep[df_cod_anomal_sep.cod_anomal.notnull()]

In [ ]:
df_cod_anomal_sep.head()

In [ ]:
df_cod_anomal_sep.info()

In [ ]:
df_cod_anomal_sep.to_csv('d_anomalias.csv', sep=';', index=False)

# 🎲 DIMENSÃO ANOMALIAS

In [ ]:
cols = ['SUBCAT', 'CLASSIF', 'RESTRSEXO', 'CAUSAOBITO', 'DESCRICAO']
df_anomal = pd.read_csv('CID-10-SUBCATEGORIAS.csv', usecols= cols, sep =';', encoding='ISO-8859-1')

In [ ]:
#renomeando colunas
novo_nome = {
    'SUBCAT': 'subcat',
    'CLASSIF': 'classif',
    'RESTRSEXO': 'restr_sexo',
    'CAUSAOBITO': 'causa_obito',
    'DESCRICAO': 'descricao_subcat',
    'DESCRABREV': 'desc_abrev_subcat'
}

df_anomal.rename(columns=novo_nome, inplace=True)

In [ ]:
#criando a chave de categoria para facilitar a análise
df_anomal['cat'] = df_anomal['subcat'].str.slice(0,3)

In [ ]:
#importando base de categorias
cols = ['CAT', 'DESCRICAO']
df_cat_anomal = pd.read_csv('CID-10-CATEGORIAS.csv', usecols= cols, sep =';', encoding='ISO-8859-1')

In [ ]:
#renomeando colunas
novo_nome = {
    'CAT': 'cat',
    'DESCRICAO': 'descricao_cat'
}

df_cat_anomal.rename(columns=novo_nome, inplace=True)

In [ ]:
#fazendo o join entre subcategoria e categoria
df_anomal = pd.merge(df_anomal,df_cat_anomal, on=['cat'], how = 'left')

In [ ]:
df_anomal.head()

In [ ]:
df_anomal.info()

In [ ]:
df_anomal.to_csv('d_desc_anomal.csv', sep = ';', index=False)

# 🎲 DIMENSAO MUNICIPIO

In [11]:
#intervalo2 = pd.read_excel(url_planilha, sheet_name='emissoes_C02', usecols= 'A:D', nrows=10)
planilha_muni = r'C:\Users\carol\OneDrive\Estudos\MBA Data Science\TCC\SINASC\RELATORIO_DTB_BRASIL_MUNICIPIO.xls'

cols = ['UF', 'Nome_UF', 'Município' ,'Código Município Completo', 'Nome_Município']
df_muni = pd.read_excel(planilha_muni, sheet_name='DTB_2022_Municipio', usecols= cols, skiprows=6, nrows=5577)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [12]:
#renomeando colunas
novo_nome = {
    'UF': 'cod_uf',
    'Nome_UF': 'uf',
    'Município': 'cod_muni',
    'Código Município Completo': 'cod_muni_completo',
    'Nome_Município': 'nome_muni'
}

df_muni.rename(columns=novo_nome, inplace=True)

In [13]:
#transformando col de cod para object
col_object1 = ['cod_uf', 'cod_muni', 'cod_muni_completo']
df_muni[col_object1] = df_muni[col_object1].astype(str)    

In [14]:
#transformando chave muni
df_muni['cod_muni'] = df_muni['cod_muni'].astype(str).str.pad(5, side = 'left', fillchar = '0')
#retirando 1 caracter da coluna cod_municompleto
df_muni['cod_muni_completo'] = df_muni['cod_muni_completo'].str.slice(0,6)

In [15]:
#criando coluna de regiao

# mapeia os estados para suas respectivas regiões
regiao_por_estado = {
    'Norte': ['Acre', 'Amapá', 'Amazonas', 'Pará', 'Rondônia', 'Roraima', 'Tocantins'],
    'Nordeste': ['Alagoas', 'Bahia', 'Ceará', 'Maranhão', 'Paraíba', 'Pernambuco', 'Piauí', 'Rio Grande do Norte', 'Sergipe'],
    'Centro-Oeste': ['Distrito Federal', 'Goiás', 'Mato Grosso', 'Mato Grosso do Sul'],
    'Sudeste': ['Espírito Santo', 'Minas Gerais', 'Rio de Janeiro', 'São Paulo'],
    'Sul': ['Paraná', 'Rio Grande do Sul', 'Santa Catarina']
}

# Função para mapear estados para regiões
def mapear_regiao(uf):
    for regiao, estados in regiao_por_estado.items():
        if uf in estados:
            return regiao
    return None  # Retorna None se o estado não for encontrado

#aplica na coluna
df_muni['regiao'] = df_muni['uf'].apply(mapear_regiao)

In [8]:
df_muni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cod_uf             5570 non-null   object
 1   uf                 5570 non-null   object
 2   cod_muni           5570 non-null   object
 3   cod_muni_completo  5570 non-null   object
 4   nome_muni          5570 non-null   object
 5   regiao             5570 non-null   object
dtypes: object(6)
memory usage: 261.2+ KB


In [9]:
df_muni.head()


,cod_uf,uf,cod_muni,cod_muni_completo,nome_muni,regiao
0,11,Rondônia,00015,110001,Alta Floresta D'Oeste,Norte
1,11,Rondônia,00379,110037,Alto Alegre dos Parecis,Norte
2,11,Rondônia,00403,110040,Alto Paraíso,Norte
3,11,Rondônia,00346,110034,Alvorada D'Oeste,Norte
4,11,Rondônia,00023,110002,Ariquemes,Norte


In [10]:
df_muni.to_csv('d_muni.csv', sep = ';', index=False)

# 🎲 FATO POPULAÇÃO

In [ ]:
cols = ['COD. UF', 'COD. MUNIC', 'POPULAÇÃO ESTIMADA']
df_pop = pd.read_excel('POP2021_20230710.xls', sheet_name='Municípios', usecols= cols, skiprows=1, nrows=5572)

In [ ]:
#renomeando colunas
novo_nome = {
    'COD. UF': 'cod_uf',
    'COD. MUNIC': 'cod_muni',
    'POPULAÇÃO ESTIMADA': 'pop_estimada'
}

df_pop.rename(columns=novo_nome, inplace=True)

In [ ]:
#alterando tipo de colunas
#transformando col de cod para object
col_object1 = ['cod_uf', 'cod_muni']
df_pop[col_object1] = df_pop[col_object1].fillna(0).astype(int).astype(str)

In [ ]:
#acrescentando zeros à esquerda do código do municipio
df_pop['cod_muni'] = df_pop['cod_muni'].str.pad(5, side = 'left', fillchar = '0')

In [ ]:
#tirando . e ou vírgula dos campos
def limpar_string(texto):
    # Verificar se o valor é uma string
    if isinstance(texto, str):
        # Define o padrão regex para encontrar parênteses e seu conteúdo
        padrao = r'\([^)]*\)'
        
        # Substitui o padrão por uma string vazia
        texto_limpo = re.sub(padrao, '', texto)
        
        # Remove pontos e vírgulas
        texto_limpo = texto_limpo.replace('.', '').replace(',', '')
        
        return texto_limpo
    else:
        return texto

df_pop['pop_estimada'] = df_pop['pop_estimada'].apply(limpar_string)

In [ ]:
#transformando a coluna de população para int
df_pop['pop_estimada'] = df_pop['pop_estimada'].fillna(0).astype(int)

In [ ]:
df_pop.head()

In [ ]:
df_pop.info()

In [ ]:
df_pop.to_csv('d_pop.csv', sep = ';', index=False)

# 🎲 DIMENSAO ESTABELECIMENTOS

In [ ]:
cols = ['CO_CNES','CO_IBGE','NO_RAZAO_SOCIAL','NO_FANTASIA','TP_GESTAO','NU_LATITUDE','NU_LONGITUDE','DS_TURNO_ATENDIMENTO','CO_NATUREZA_JUR','ST_CENTRO_CIRURGICO','ST_CENTRO_OBSTETRICO','ST_CENTRO_NEONATAL','ST_ATEND_HOSPITALAR','ST_ATEND_AMBULATORIAL','CO_AMBULATORIAL_SUS']

df_estab = pd.read_csv('cnes_estabelecimentos.csv', usecols= cols, sep =';', encoding='ISO-8859-1')

In [ ]:
#renomeando colunas
novo_nome = {

    'CO_CNES': 'co_cnes',
    'CO_IBGE': 'co_ibge',
    'NO_RAZAO_SOCIAL': 'no_razao_social',
    'NO_FANTASIA': 'no_fantasia',
    'TP_GESTAO': 'tp_gestao',
    'NU_LATITUDE': 'nu_latitude',
    'NU_LONGITUDE': 'nu_longitude',
    'DS_TURNO_ATENDIMENTO': 'ds_turno_atendimento',
    'CO_NATUREZA_JUR': 'co_natureza_jur',
    'ST_CENTRO_CIRURGICO': 'st_centro_cirurgico',
    'ST_CENTRO_OBSTETRICO': 'st_centro_obstetrico',
    'ST_CENTRO_NEONATAL': 'st_centro_neonatal',
    'ST_ATEND_HOSPITALAR': 'st_atend_hospitalar',
    'ST_ATEND_AMBULATORIAL': 'st_atend_ambulatorial',
    'CO_AMBULATORIAL_SUS': 'co_ambulatorial_sus'
    
}

df_estab.rename(columns=novo_nome, inplace=True)

In [ ]:
df_estab.info()

In [ ]:
df_estab['co_natureza_jur'] = df_estab['co_natureza_jur'].fillna(0).astype(int)

col_object = ['co_cnes','co_ibge','no_razao_social','no_fantasia','tp_gestao','ds_turno_atendimento','co_natureza_jur','st_centro_cirurgico','st_centro_obstetrico','st_centro_neonatal','st_atend_hospitalar','st_atend_ambulatorial','co_ambulatorial_sus']
df_estab[col_object] = df_estab[col_object].astype(str)

In [ ]:
#nova coluna de natureza do estabelecimento
df_estab['co_natureza_jur_agg'] = df_estab['co_natureza_jur'].str.slice(0,1)

In [ ]:
#trazendo a marcação da categoria no df seguinto clasificação do ibge
mapeamento = {'1': 'administracao publica', '2':'entidades empresariais', '3':'entidades sem fins lucrativos', '4':'pessoas físicas', '5': 'organizações internacionais e outras instituições extraterritoriais' }
df_estab['classe_natureza'] = df_estab['co_natureza_jur_agg'].map(mapeamento).fillna('sem info')

In [ ]:
df_estab.head()

In [ ]:
df_estab.to_csv('d_estab.csv', sep = ';', index=False)

In [ ]:
#nível de correspondência dos cod de estabelecimento na base sinasc -> fazer cod estab únicos ja´em string
#dps match com essa base pra ver qnt tem qnt nãotem